In [14]:
import cv2
import os

INPUT_DIR = "fotos"
OUTPUT_DIR = "fotos_binarizadas"

os.makedirs(OUTPUT_DIR, exist_ok=True)

for subdir, _, files in os.walk(INPUT_DIR):
    rel_path = os.path.relpath(subdir, INPUT_DIR)
    out_subdir = os.path.join(OUTPUT_DIR, rel_path)
    os.makedirs(out_subdir, exist_ok=True)

    for file in files:
        if file.lower().endswith(".jpg"):
            in_path = os.path.join(subdir, file)
            out_path = os.path.join(out_subdir, file)

            img = cv2.imread(in_path)

            if img is None:
                print(f"⚠️ No se pudo leer {in_path}")
                continue

            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            _, binary = cv2.threshold(
                gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU
            )

            cv2.imwrite(out_path, binary)
            print(f"✅ Procesada: {in_path} -> {out_path}")

print("✨ Proceso completado")

✅ Procesada: fotos/146c3702-37e7-402e-9be2-22079002f881/14.jpg -> fotos_binarizadas/146c3702-37e7-402e-9be2-22079002f881/14.jpg
✅ Procesada: fotos/146c3702-37e7-402e-9be2-22079002f881/15.jpg -> fotos_binarizadas/146c3702-37e7-402e-9be2-22079002f881/15.jpg
✅ Procesada: fotos/146c3702-37e7-402e-9be2-22079002f881/17.jpg -> fotos_binarizadas/146c3702-37e7-402e-9be2-22079002f881/17.jpg
✅ Procesada: fotos/146c3702-37e7-402e-9be2-22079002f881/16.jpg -> fotos_binarizadas/146c3702-37e7-402e-9be2-22079002f881/16.jpg
✅ Procesada: fotos/146c3702-37e7-402e-9be2-22079002f881/13.jpg -> fotos_binarizadas/146c3702-37e7-402e-9be2-22079002f881/13.jpg
✅ Procesada: fotos/146c3702-37e7-402e-9be2-22079002f881/18.jpg -> fotos_binarizadas/146c3702-37e7-402e-9be2-22079002f881/18.jpg
✅ Procesada: fotos/146c3702-37e7-402e-9be2-22079002f881/19.jpg -> fotos_binarizadas/146c3702-37e7-402e-9be2-22079002f881/19.jpg
✅ Procesada: fotos/9fe3e8a4-30e9-429d-98cb-8793cb6a4bb3/28.jpg -> fotos_binarizadas/9fe3e8a4-30e9-429d-9

In [15]:
import cv2, os, numpy as np
from math import pi

INPUT_DIR = "fotos"
OUTPUT_DIR = "fotos_binarizadas_limpias"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# parámetros
LEFT_MASK_FRAC = 0.08        # enmascara el 8% izquierdo (cámbialo si hay borde brillante)
BLUR_KSIZE = 7               # suavizado para reducir ruido (impar)
OPEN_K = 5                   # apertura morfológica para quitar puntos sueltos
AREA_MIN = 150               # descarta blobs muy pequeños
CIRC_MIN = 0.5               # circularidad mínima 4πA/P^2 (0..1)

def keep_largest_circular(blob_mask):
    # devuelve una máscara con el blob más grande y más circular
    cnts, _ = cv2.findContours(blob_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    best = None
    best_score = 0.0
    h, w = blob_mask.shape
    for c in cnts:
        area = cv2.contourArea(c)
        if area < AREA_MIN: 
            continue
        peri = cv2.arcLength(c, True) or 1.0
        circ = (4.0 * pi * area) / (peri * peri)  # 1.0 = círculo perfecto
        score = area * (0.5 + 0.5 * min(max(circ, 0.0), 1.0))  # pondera por circularidad y área
        if circ >= CIRC_MIN and score > best_score:
            best = c
            best_score = score
    out = np.zeros((h, w), np.uint8)
    if best is not None:
        cv2.drawContours(out, [best], -1, 255, thickness=-1)
    return out

for subdir, _, files in os.walk(INPUT_DIR):
    rel = os.path.relpath(subdir, INPUT_DIR)
    out_sub = os.path.join(OUTPUT_DIR, rel)
    os.makedirs(out_sub, exist_ok=True)

    for f in files:
        if not f.lower().endswith(".jpg"):
            continue
        in_path  = os.path.join(subdir, f)
        out_path = os.path.join(out_sub, f)

        img = cv2.imread(in_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print("⚠️ No se pudo leer", in_path); continue

        h, w = img.shape

        # (1) Enmascara franja izquierda si hay luz parásita
        if LEFT_MASK_FRAC > 0:
            m = int(w * LEFT_MASK_FRAC)
            bg = int(img[:, m:m+20].mean()) if m+20 < w else int(img.mean())
            img[:, :m] = bg

        # (2) Suavizado para reducir ruido
        blur = cv2.GaussianBlur(img, (BLUR_KSIZE, BLUR_KSIZE), 0)

        # (3) Otsu (la pelota es brillante → queremos binario con pelota blanca)
        _, bin0 = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # (4) Morfología: apertura para quitar puntos sueltos
        k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (OPEN_K, OPEN_K))
        opened = cv2.morphologyEx(bin0, cv2.MORPH_OPEN, k, iterations=1)

        # (5) Quédate con el blob más grande y circular (la pelota)
        clean = keep_largest_circular(opened)

        cv2.imwrite(out_path, clean)
        print(f"✅ {in_path} -> {out_path}")

print("✨ Proceso completado")

✅ fotos/146c3702-37e7-402e-9be2-22079002f881/14.jpg -> fotos_binarizadas_limpias/146c3702-37e7-402e-9be2-22079002f881/14.jpg
✅ fotos/146c3702-37e7-402e-9be2-22079002f881/15.jpg -> fotos_binarizadas_limpias/146c3702-37e7-402e-9be2-22079002f881/15.jpg
✅ fotos/146c3702-37e7-402e-9be2-22079002f881/17.jpg -> fotos_binarizadas_limpias/146c3702-37e7-402e-9be2-22079002f881/17.jpg
✅ fotos/146c3702-37e7-402e-9be2-22079002f881/16.jpg -> fotos_binarizadas_limpias/146c3702-37e7-402e-9be2-22079002f881/16.jpg
✅ fotos/146c3702-37e7-402e-9be2-22079002f881/13.jpg -> fotos_binarizadas_limpias/146c3702-37e7-402e-9be2-22079002f881/13.jpg
✅ fotos/146c3702-37e7-402e-9be2-22079002f881/18.jpg -> fotos_binarizadas_limpias/146c3702-37e7-402e-9be2-22079002f881/18.jpg
✅ fotos/146c3702-37e7-402e-9be2-22079002f881/19.jpg -> fotos_binarizadas_limpias/146c3702-37e7-402e-9be2-22079002f881/19.jpg
✅ fotos/9fe3e8a4-30e9-429d-98cb-8793cb6a4bb3/28.jpg -> fotos_binarizadas_limpias/9fe3e8a4-30e9-429d-98cb-8793cb6a4bb3/28.jpg
